In [11]:
#(i)Compute the Mean and Standard Deviation
import numpy as np

def compute_mean_std(data):
    """
    Compute the mean and standard deviation of a list of numerical data.
    
    Parameters:
    data (list): A list of numerical values.
    
    Returns:
    mean (float): The mean of the data.
    std (float): The standard deviation of the data.
    """
    n = len(data)
    mean = sum(data) / n
    variance = sum((x - mean) ** 2 for x in data) / (n - 1)  # Sample variance
    std = np.sqrt(variance)
    return mean, std

#(ii)Compute t0
def compute_t0(data, mu0):
    """
    Compute the t-statistic t0 for a given dataset and hypothesized mean.
    
    Parameters:
    data (list): A list of numerical values.
    mu0 (float): The hypothesized mean (null hypothesis).
    
    Returns:
    t0 (float): The t-statistic.
    """
    mean, std = compute_mean_std(data)
    n = len(data)
    t0 = (mean - mu0) / (std / np.sqrt(n))
    return t0
    
#(iii)Find t* Using Numerical Integration
import numpy as np

def gamma_function(x):
    """
    Compute the gamma function using Stirling's approximation and recursion.
    
    Parameters:
    x (float): The input to the gamma function.
    
    Returns:
    float: The value of the gamma function at x.
    """
    if x < 0:
        raise ValueError("Gamma function is not defined for negative integers.")
    if x == 0:
        return np.inf
    if x < 10:
        # Use recursion for small x: Γ(x) = Γ(x + 1) / x
        return gamma_function(x + 1) / x
    else:
        # Stirling's approximation for large x
        return np.sqrt(2 * np.pi / x) * (x / np.e) ** x * (1 + 1 / (12 * x) + 1 / (288 * x**2))

def t_distribution_pdf(x, nu):
    """
    Compute the probability density of the t-distribution at a given point x.
    
    Parameters:
    x (float): The point at which to evaluate the density.
    nu (int): The degrees of freedom of the t-distribution.
    
    Returns:
    density (float): The probability density at point x.
    """
    coeff = gamma_function((nu + 1) / 2) / (np.sqrt(nu * np.pi) * gamma_function(nu / 2))
    density = coeff * (1 + x**2 / nu) ** (-0.5 * (nu + 1))
    return density

def find_t_star(prob, nu, x_start=0, x_end=20, num_points=10000):
    """
    Find the t-value t* for a given cumulative probability and degrees of freedom.
    
    Parameters:
    prob (float): The cumulative probability (between 0 and 1).
    nu (int): The degrees of freedom of the t-distribution.
    x_start (float): The start point for numerical integration.
    x_end (float): The end point for numerical integration.
    num_points (int): The number of points to use in the numerical integration.
    
    Returns:
    float: The t-value t* such that the area between [-t*, t*] equals the given probability.
    """
    # Define the x values
    x = np.linspace(x_start, x_end, num_points)
    
    # Apply the density function to the x values
    y = t_distribution_pdf(x, nu)
    
    # Numerical integration to compute the CDF
    cdf = np.cumsum(y) * (x[1] - x[0])
    
    # Find the t-value where the cumulative probability reaches half of the required probability
    target_half_prob = prob / 2
    index = np.where(cdf >= target_half_prob)[0][0]
    
    return x[index]

#(iv)test whether t0E[-t*,t*]
def is_t0_in_interval(t0, t_star):
    """
    Check if t0 falls within the interval [-t*, t*].
    
    Parameters:
    t0 (float): The computed t-statistic.
    t_star (float): The critical t-value.
    
    Returns:
    bool: True if t0 is in [-t*, t*], False otherwise.
    """
    return abs(t0) <= t_star

#(v)Apply the t-Test to the Given Data
# Given data
data = [92.64, 79.00, 84.79, 97.41, 93.68, 65.23, 84.50, 73.49, 73.97, 79.11]
mu0 = 75  # National average

# Step 1: Compute mean and standard deviation
mean, std = compute_mean_std(data)
print(f"Mean: {mean}, Standard Deviation: {std}")

# Step 2: Compute t0
t0 = compute_t0(data, mu0)
print(f"t0: {t0}")

# Step 3: Find t*
nu = len(data) - 1  # Degrees of freedom
t_star = find_t_star(0.95, nu)
print(f"t*: {t_star}")

# Step 4: Check if t0 is in [-t*, t*]
result = is_t0_in_interval(t0, t_star)
print(f"t0 ∈ [-t*, t*]: {result}")

# Step 5: Conclusion
if result:
    print("We fail to reject the null hypothesis. There is no significant difference in the average score.")
else:
    print("We reject the null hypothesis. There is a significant difference in the average score.")

Mean: 82.382, Standard Deviation: 10.193467189005581
t0: 2.290087686017293
t*: 2.2522252225222523
t0 ∈ [-t*, t*]: False
We reject the null hypothesis. There is a significant difference in the average score.
